In [2]:
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time


#get the thresholds
b = pd.read_csv("data/WorkingData.csv")
series = (b['ContestId']);
series2 = (b['MaxNumberPlayers'])

#allocate the final data array
final = []

#tic
t = time.time()

#open the file
with open('data/JacksonData_01_29.csv', newline='') as f:
    #prime the reader
    reader = csv.reader(f)
    header = next(reader)
    
    for row in reader:
        
        if (row[2]==row[6] and row[0]!='0'): #if minutes left and minute length match
            
            ##############
            #fill in time series
            ##############
            s_log_entries = pd.Series(log_entries); #make a series out of previous sparse time series
            s_log_entries = s_log_entries.interpolate(); #interpolate
            
            ##############
            #linear regression
            ##############
            seen_entries = s_log_entries[:-240]; #give it the data we see 4 hours out
            
            #initialize W
            #forget_factor = 1; #set the value for the diagonal
            #W = np.identity(len(seen_entries))*forget_factor;

            #initialize H
            H1 = np.linspace(1,1,len(seen_entries));
            H2 = np.linspace(0,len(seen_entries)-1, len(seen_entries));
            H = np.vstack((H1,H2));
            H = np.transpose(H);

            #calculate alpha and beta
            #if (np.linalg.det(np.transpose(H).dot(W).dot(H))!=0):
            if (np.linalg.det(np.transpose(H).dot(H))!=0):
                [alpha, beta] = np.linalg.inv(np.transpose(H).dot(H)).dot(np.transpose(H)).dot(seen_entries);
                #[alpha, beta] = np.linalg.inv(np.transpose(H).dot(W).dot(H)).dot(np.transpose(H)).dot(W).dot(seen_entries);

            else:
                [alpha, beta] = [0,0];
            
            
            #################
            #comparison to threshold
            #################
            ind = series[series == int(row1[1])].index[0];
            thresh = math.log(series2[ind]);
            if ((alpha + beta*(int(row1[2]))) >= thresh):
                pred_succ = 1;
            else:
                pred_succ = 0;
            
            
            #################
            #gather and write data
            #################
            line = [row1[1], row1[3], alpha, beta, pred_succ]; #write the line
            final.append(line); #append the line to our final array
        
            #################
            #start new time series
            #################
            log_entries = [np.nan]*(int(row[2])+1); 
            log_entries[0] = math.log(int(row[5]));
            
            
            
        elif(row[2]==row[6] and row[0]=='0'): #if we are at the first line of the whole thing
            #just initialize and fill first
            log_entries = [np.nan]*(int(row[2])+1);
            log_entries[0] = math.log(int(row[5]));
            
            
        else: #we're just at a regular line
            row1 = row;
            slot = int(row[2]) - int(row[6]);
            log_entries[slot] = math.log(int(row[5]));
            


f.close()       
elapsed = time.time()-t
elapsed

1289.5261158943176

In [3]:
df = pd.DataFrame(final, columns = ['ContestId', 'Success', 'alpha', 'beta', 'Pred_Success'])
df.to_csv(path_or_buf = 'linreg_results_0204.csv')

In [83]:
[TP,FP,TN,FN] = [0,0,0,0];
for i in range(len(final)):
    if final[i][1]=='False' and final[i][4]==0:
        TN = TN + 1;
    elif final[i][1]=='False' and final[i][4]==1:
        FP = FP + 1;
    elif final[i][1]=='True' and final[i][4]==0:
        FN = FN + 1;
    elif final[i][1]=='True' and final[i][4]==1:
        TP = TP + 1;
print(TN,TP,FN,FP)
rate = (TN + TP)/len(final)
print(rate)

17303 167195 68172 7870
0.7081369463422124
